In [1]:
import numpy as np
from importlib import reload
import interpolate_fhat_Jhat as intf
import matplotlib.pyplot as plt
import calculate_Gamma 
from scipy.interpolate import interp1d, splev, splrep
from statistics import stdev
from scipy.optimize import curve_fit

### Data and LCDM prediction

In [2]:
# Data as described in 2403.13709
z_data = np.array([0.001, 0.02, 0.02, 0.02, 0.15, 0.17, 0.18, 0.38, 0.38, 0.51, 0.70, 0.85, 0.60, 0.86, 1.40, 0.44, 0.60, 0.73, 0.978, 1.23, 1.526, 1.944])
fhat_data = np.array([0.505, 0.428, 0.398, 0.314, 0.530, 0.51, 0.36, 0.44, 0.497, 0.459, 0.473, 0.315, 0.55, 0.4, 0.482, 0.413, 0.390, 0.437, 0.379, 0.385, 0.342, 0.364])
fhat_data_errors = np.array([0.085, 0.0465, 0.065, 0.048, 0.16, 0.06, 0.09, 0.06, 0.045, 0.038, 0.041, 0.095, 0.12, 0.11, 0.116, 0.08, 0.063, 0.072, 0.176, 0.099, 0.07, 0.106])

In [3]:
# Building the full covariance, taking the one of WiggleZ measurements and of SDSS-IV quasars into account
fhat_cov_partial = np.diag(fhat_data_errors[:len(z_data)-7]**2)

covariance_wiggle = 1e-3*np.array([[6.400, 2.570, 0.000],
                                   [2.570, 3.969, 2.540], 
                                   [0.000, 2.540, 5.184]])
covariance_SDSS = 1e-2*np.array([[3.098, 0.892, 0.329, -0.021],
                                 [0.892, 0.980, 0.436, 0.076],
                                 [0.329, 0.436, 0.490, 0.350],
                                 [-0.021, 0.076, 0.350, 1.124]])

fhat_data_covariance = np.block([[fhat_cov_partial, np.zeros([np.shape(fhat_cov_partial)[0],7])],
                             [np.zeros([3,np.shape(fhat_cov_partial)[0]]),covariance_wiggle, np.zeros([3,4])],
                             [np.zeros([4, np.shape(fhat_cov_partial)[0]+3]),covariance_SDSS]])

fhat_data_current = {
    'z' : z_data,
    'mean' : fhat_data,
    'cov' : fhat_data_covariance
}

In [4]:
# Jhat baseline Polychord
maglim_zeff=np.array([0.295, 0.467, 0.626, 0.771])
Jhat_maglim=np.array([0.32546732, 0.33277392, 0.38656369, 0.35394343])
Jhat_covariance = np.array([[2.22596522e-04, 7.44557424e-05, 5.89004236e-05, 7.71102354e-05],
 [7.44557424e-05, 3.37844968e-04, 1.05329570e-04, 1.38623775e-04],
 [5.89004236e-05, 1.05329570e-04, 7.55843341e-04, 3.65660247e-04],
 [7.71102354e-05, 1.38623775e-04, 3.65660247e-04, 1.24053020e-03]])

In [5]:
Jhat_errors = np.sqrt(np.diag(Jhat_covariance))

In [6]:
Jhat_reco = {
    'mean' : Jhat_maglim,
    'cov' : Jhat_covariance,
}

In [7]:
z_range=np.linspace(0,2,100)
# Calculate the function values at the z values
fhat_values = calculate_Gamma.fhat_LCDM(z_range)

# Numerically approximate the derivative of fhat_LCDM(z)
fhat_derivative = np.gradient(fhat_values, z_range)

In [8]:
Jhat_values = calculate_Gamma.Jhat_LCDM(z_range, Omega_m0 = 0.328, sigma_80 = 0.852)

# Numerically approximate the derivative of Jhat_LCDM(z)
Jhat_derivative = np.gradient(Jhat_values, z_range)

## Interpolation of fhat

In [9]:
#Using maglim_zeff as node values
[best_fit_values_maglim, covariance_matrix_maglim] = intf.minuit_spline(maglim_zeff,z_data = z_data, fhat_data = fhat_data, fhat_data_errors = fhat_data_errors, fhat_data_covariance = fhat_data_covariance, use_covariance = True)
fhat_reco = intf.spline_interpolation(best_fit_values_maglim, covariance_matrix_maglim, return_covariance = True)

In [10]:
#Other node values
#node_value2 = np.array([0.2,1.0,1.5])
node_value2 = np.array([0.2,1.0,1.5])
[best_fit_values_maglim2, covariance_matrix_maglim2] = intf.minuit_spline(node_value2, z_data = z_data, fhat_data = fhat_data, fhat_data_errors = fhat_data_errors, fhat_data_covariance = fhat_data_covariance, use_covariance = True)
fhat_reco2 = intf.spline_interpolation(best_fit_values_maglim2, covariance_matrix_maglim2, return_covariance = True, zarray = node_value2)

In [11]:
#Get derivatives and covariance of derivatives at maglim zeff
fhat_reco_maglim = intf.spline_interpolation(best_fit_values_maglim, covariance_matrix_maglim, zarray=maglim_zeff, z_range = maglim_zeff, return_covariance = True)

In [12]:
#Get derivatives and covariance of derivatives at maglim zeff
fhat_reco_maglim2 = intf.spline_interpolation(best_fit_values_maglim2, covariance_matrix_maglim2, zarray=node_value2, z_range = maglim_zeff, return_covariance = True)

In [13]:
variances_maglim = np.diag(fhat_reco_maglim['cov'])
std_devs_maglim = np.sqrt(variances_maglim)

In [14]:
variances_maglim2 = np.diag(fhat_reco_maglim2['cov'])
std_devs_maglim2 = np.sqrt(variances_maglim2)

In [15]:
variances_all = np.diag(fhat_reco['cov'])
std_devs_all = np.sqrt(variances_all)

In [16]:
variances_all2 = np.diag(fhat_reco2['cov'])
std_devs_all2 = np.sqrt(variances_all2)

#### Akaike and Bayesian Information Criteria

In [17]:
def IC(best_fit_values, zarray):
    chi_squared = intf.chi_squared_spline(best_fit_values, zarray, z_data = z_data, fhat_data = fhat_data, 
                                          fhat_data_covariance = fhat_data_covariance, use_covariance = True)
    AIC = 2*len(zarray)+2*chi_squared
    BIC= np.log(len(z_data))*len(zarray)+2*chi_squared
    return [chi_squared, AIC, BIC]   

In [18]:
AIC_values=[]
BIC_values=[]
chi_squared_values=[]

for n in range(3,10):

    # Calculate percentiles
    percentiles = np.linspace(0, 100, n+2)

    # Find values corresponding to percentiles
    node_values = np.percentile(z_data, percentiles)[1:n+1]
    #z_test=np.linspace(0.25,1.0,n)
    [best_fit_values_test, covariance_matrix_test] = intf.minuit_spline(node_values, z_data = z_data, fhat_data = fhat_data, 
                                          fhat_data_covariance = fhat_data_covariance, use_covariance = True)
    [chi_squared, AIC, BIC]=IC(best_fit_values_test, node_values)
    chi_squared_values.append(chi_squared/(22-n))
    AIC_values.append(AIC)
    BIC_values.append(BIC)
    
print(chi_squared_values)
print(AIC_values)
print(BIC_values)

[0.8498833809425145, 0.7723833458840795, 0.7921296507766393, 0.8392106764535627, 0.8864118578412501, 0.9451649687204933, 0.9459011450588404]
[38.29556847581555, 35.80580045182686, 36.93240812640573, 38.85474164651401, 40.592355735237504, 42.46461912417381, 42.59342977152985]
[41.568695835890495, 40.169970265260126, 42.38762039319732, 45.400996366663904, 48.22965290874572, 51.19295875104034, 52.4128118517547]


#### Testing the fifth force

In [19]:
Hprime = calculate_Gamma.Hprime(maglim_zeff)
H = calculate_Gamma.E(maglim_zeff)/(1+maglim_zeff)
fprime = -(1+maglim_zeff)*fhat_reco_maglim['mean_d1']
fprime2 = -(1+maglim_zeff)*fhat_reco_maglim2['mean_d1']

In [20]:
Gamma = 2/3*np.array(best_fit_values_maglim/Jhat_maglim)*(Hprime/H+1+fprime/best_fit_values_maglim)
Gamma2 = 2/3*np.array(fhat_reco_maglim2['mean']/Jhat_maglim)*(Hprime/H+1+fprime2/fhat_reco_maglim2['mean'])

In [21]:
[Gamma, covariance_matrix_gamma, dlnf_mean, covariance_dlnf]=calculate_Gamma.Gamma_reco(fhat_reco_maglim, Jhat_reco, include_cov01 = True)

AttributeError: 'numpy.ndarray' object has no attribute 'Gamma_reco'

In [ ]:
[Gamma2, covariance_matrix_gamma2, dlnf_mean2, covariance_dlnf2]=calculate_Gamma.Gamma_reco(fhat_reco_maglim2, Jhat_reco, include_cov01 = True)

In [ ]:
stdev_Gamma = np.sqrt(np.diag(covariance_matrix_gamma))
stdev_Gamma2 = np.sqrt(np.diag(covariance_matrix_gamma2))

In [ ]:
[dlnf, dlnf_cov] = calculate_Gamma.dlnf_dln1pz(fhat_reco)
err_dlnf = np.sqrt(np.diag(dlnf_cov))

In [ ]:
[dlnf2, dlnf_cov2] = calculate_Gamma.dlnf_dln1pz(fhat_reco2)
err_dlnf2 = np.sqrt(np.diag(dlnf_cov2))

In [27]:
[dlnf_maglim, dlnf_cov_maglim] = calculate_Gamma.dlnf_dln1pz(fhat_reco_maglim)
err_dlnf_maglim = np.sqrt(np.diag(dlnf_cov_maglim))

AttributeError: 'numpy.ndarray' object has no attribute 'dlnf_dln1pz'

In [23]:
dlnf_LCDM = fhat_derivative * (1+z_range)/fhat_values

#### Plots for fifth force paper (current constraints)

In [ ]:
plt.figure(figsize=(6, 5))

# Plot spline interpolation
fill1 = plt.fill_between(
    fhat_reco['x'],
    dlnf - err_dlnf,
    dlnf + err_dlnf,
    color='C0',
    alpha=0.3,
    label='4 knots'
)

#Plot inferred values
plt.errorbar(maglim_zeff, dlnf_maglim, fmt = '.', yerr = err_dlnf_maglim, 
             label='MagLim redshifts', color = 'red', linewidth = 2)


plt.xlabel("$z$", fontsize=16)
plt.ylabel('$\\mathrm{d}\,\ln\hat{f}(z)/\\mathrm{d}\,\ln(1+z)$', fontsize = 16)
plt.xlim(0.0,2.0)
plt.ylim(-1.2,1.2)



line = plt.plot(z_range, dlnf_LCDM, color='black', label='No fifth force', linewidth=1)[0]

plt.savefig("dlnf.png", dpi=600, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6, 5))

fill1 = plt.fill_between(
    fhat_reco['x'],
    dlnf - err_dlnf,
    dlnf + err_dlnf,
    color='C0',
    alpha=0.3,
    label='4 knots'
)

fill2 = plt.fill_between(
    fhat_reco2['x'],
    dlnf2 - err_dlnf2,
    dlnf2 + err_dlnf2,
    color='orange',
    alpha=0.3,
    label='3 knots'
)



plt.xlabel("$z$", fontsize=16)
plt.ylabel('$\\mathrm{d}\,\ln\hat{f}(z)/\\mathrm{d}\,\ln(1+z)$', fontsize = 16)
plt.xlim(0.0,2.0)
plt.ylim(-1.2,1.2)

errbar = plt.errorbar([1], [-10], yerr=[0.01], fmt='.', label='Measured $f\sigma_8$ data',
                      color='black', elinewidth=1)

line = plt.plot(z_range, dlnf_LCDM, color='black', label='No fifth force', linewidth=1)[0]


plt.savefig("dlnf_3_4.png", dpi=600, bbox_inches='tight')

In [ ]:
plt.figure(figsize=(6, 5))
#Plot the LCDM prediction
zval=np.linspace(0,2,100)
fhat_LCDM = calculate_Gamma.fhat_LCDM(zval) 
plt.plot(zval, fhat_LCDM, label='No fifth force', color = 'black', linewidth = 1)


#Plot measured data
plt.errorbar(z_data, fhat_data, yerr = fhat_data_errors, fmt = '.', label = 'Measured data', color = 'black', elinewidth=1)
    

#Plot reconstructed functions

plt.fill_between(fhat_reco['x'], fhat_reco['mean']+std_devs_all, fhat_reco['mean']-std_devs_all, 
                 color='C0', alpha = 0.3, label='Spline reconstruction')
        

#Plot inferred values
plt.errorbar(maglim_zeff, fhat_reco_maglim['mean'], fmt = '.', yerr = std_devs_maglim, 
             label='MagLim redshifts', color = 'red', linewidth = 2)


ticks = [0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00]

# Set the x-axis ticks
plt.xticks(ticks,fontsize=14)
plt.yticks(fontsize=14)


plt.xlabel("$z$", fontsize=16)
plt.ylabel("$\hat{f}\,(z)$", fontsize=16)
#plt.ylim(0.1, 0.8)
plt.xlim(-0.05,2.05)
plt.legend(ncol=2, prop={'size': 9.8},  columnspacing=0.7, fontsize = 12)
#plt.title(title)
plt.tight_layout()
plt.savefig("results_fhat_interpolation_new.png",dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))
#Plot the LCDM prediction
zval=np.linspace(0,2,100)
fhat_LCDM = calculate_Gamma.fhat_LCDM(zval) 
GR = plt.plot(zval, fhat_LCDM, label='No fifth force', color = 'black', linewidth = 1)


#Plot measured data
data = plt.errorbar(z_data, fhat_data, yerr = fhat_data_errors, fmt = '.', label = 'Measured data', color = 'black', elinewidth=1)
    

#Plot reconstructed functions

fill1 = plt.fill_between(fhat_reco['x'], fhat_reco['mean']+std_devs_all, fhat_reco['mean']-std_devs_all, 
                 color='C0', alpha = 0.3, label='4 knots')

fill2 = plt.fill_between(fhat_reco2['x'], fhat_reco2['mean']+std_devs_all2, fhat_reco2['mean']-std_devs_all2, 
                 color='orange', alpha = 0.3, label='3 knots')
        
    

ticks = [0.00, 0.25, 0.50, 0.75, 1.00, 1.25, 1.50, 1.75, 2.00]

# Set the x-axis ticks
plt.xticks(ticks,fontsize=14)
plt.yticks(fontsize=14)

# Then, collect handles and labels manually
handles = [fill1, fill2, errbar, line]
labels = ['4 knots', '3 knots', 'Measured data', 'No fifth force']

# Then set the legend with desired order and spacing
plt.legend(handles, labels, fontsize=12, ncol=2)


plt.xlabel("$z$", fontsize=16)
plt.ylabel("$\hat{f}\,(z)$", fontsize=16)
#plt.ylim(0.1, 0.8)
plt.xlim(-0.05,2.05)
#plt.title(title)
plt.tight_layout()
plt.savefig("results_fhat_3_4.png",dpi=600, bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=(6, 5))

plt.plot(np.linspace(0.2,0.9,50), np.full(50, 0), color='black', linewidth=1.0)

# Plot the measured data with larger markers and wider error bars
plt.errorbar(maglim_zeff-0.005, Gamma - 1, yerr=stdev_Gamma, fmt='.', markersize=10, elinewidth=2, color = 'red', label = '4 knots')

# Plot the measured data with larger markers and wider error bars
plt.errorbar(maglim_zeff+0.005, Gamma2 - 1, yerr=stdev_Gamma2, fmt='.', markersize=10, elinewidth=2, color = 'orange', label = '3 knots')


plt.xlabel("z", fontsize=16)
plt.ylabel("$\Gamma\,(z)$", fontsize=16)
plt.xlim(0.2,0.9)
# plt.ylim(ylim_lower, ylim_upper)
plt.legend(ncol = 2, loc = 'upper center', fontsize = 12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.savefig("results_Gamma_3_4.png", dpi=600, bbox_inches='tight')

In [ ]:
def const_Gamma(z, Gamma_const_val):
    return Gamma_const_val

In [ ]:
Omm0 = 0.311
OmL0 = 1 - 0.311

def Omega_L(z):
    return OmL0/(OmL0+Omm0*(1+z)**3)

def Gamma_DE(z, Gamma0):
    return 1+Gamma0*Omega_L(z)/OmL0

In [ ]:
[Gamma0_mean, Gamma0_stdev2] = curve_fit(Gamma_DE, maglim_zeff, Gamma, sigma=covariance_matrix_gamma, absolute_sigma=True)
Gamma0_stdev = np.sqrt(Gamma0_stdev2)
print(Gamma0_mean, Gamma0_stdev)

In [ ]:
[const_Gamma_mean, const_Gamma_stdev2] = curve_fit(const_Gamma, maglim_zeff, Gamma, sigma=covariance_matrix_gamma, absolute_sigma=True)
const_Gamma_stdev = np.sqrt(const_Gamma_stdev2)
print(const_Gamma_mean, const_Gamma_stdev)

In [ ]:
plt.figure(figsize=(6, 5))

color_fit = 'forestgreen'
plt.plot(np.linspace(0.2,0.9,2), np.full(2, const_Gamma_mean-1), color=color_fit, linewidth=1.5, alpha = 0.8, label='Constant $\Gamma(z)$')
plt.fill_between([0.2,0.9], np.full(2,const_Gamma_mean-1-const_Gamma_stdev), np.full(2,const_Gamma_mean-1+const_Gamma_stdev), 
                 color=color_fit, alpha = 0.2)

plt.plot(np.linspace(0.2,0.9,50), np.full(50, 0), color='black', linewidth=1.0)

# Plot the measured data with larger markers and wider error bars
plt.errorbar(maglim_zeff, Gamma - 1, yerr=stdev_Gamma, fmt='.', markersize=10, elinewidth=2, color = 'red', label = 'MagLim redshifts')

# Include Fifth force evolving with DE density

plt.plot(np.linspace(0.2,0.9,100), Gamma_DE(np.linspace(0.2,0.9,100), Gamma0_mean)-1, color='blue', alpha = 0.8, linewidth=1.5, label = '$\Gamma(z)$ evolving with DE')
plt.plot(np.linspace(0.2,0.9,100), Gamma_DE(np.linspace(0.2,0.9,100), Gamma0_mean+Gamma0_stdev[0])-1,'--', color='blue', alpha = 0.8, linewidth=1.)
plt.plot(np.linspace(0.2,0.9,100), Gamma_DE(np.linspace(0.2,0.9,100), Gamma0_mean-Gamma0_stdev[0])-1,'--', color='blue', alpha = 0.8, linewidth=1.)


plt.xlabel("z", fontsize=16)
plt.ylabel("$\Gamma\,(z)$", fontsize=16)
plt.xlim(0.2,0.9)
# plt.ylim(ylim_lower, ylim_upper)
plt.legend(fontsize = 12, loc = 'upper left')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.savefig("results_Gamma_withDE.png", dpi=600, bbox_inches='tight')